# 銀行AI客服
## 先將QA轉成OpenAI所需要的jsonl檔案

In [18]:
import json

def convert_json_to_jsonl():
    with open('bank.json', 'r', encoding='utf-8') as input_file:
        input_json = json.load(input_file)
        # Create JSONL output structure
        output_jsonl_strs = []
        # Add QA pairs to the output JSONL structure
        for qa_pair in input_json["qa"]:
            system_message = "你是使用繁體中文的銀行客服人員"  # Define system message
            messages = {
                "messages": [ 
                    { "role": "system", "content": system_message },
                    { "role": "user", "content": qa_pair["question"] },
                    { "role": "assistant", "content": qa_pair["answer"] }
                ]
            }
            output_jsonl_strs.append(json.dumps(messages, ensure_ascii=False))

        # Write JSONL output to a file
        output_jsonl_path = "bank.jsonl"
        with open(output_jsonl_path, 'w', encoding='utf-8') as jsonl_file:
            jsonl_file.write("\n".join(output_jsonl_strs))

        return output_jsonl_path

# 调用函数以执行转换并获得JSONL文件的路径
jsonl_file_path = convert_json_to_jsonl()
print("JSONL 文件已创建:", jsonl_file_path)

JSONL 文件已创建: bank.jsonl


## 使用openai_api_key創一個fine-tuning

In [22]:
from openai import OpenAI

# OpenAI Key
api_key = "OPENAI_API_KEY"
client = OpenAI(api_key = api_key)
# 創建一個fine tune
client.files.create(
  file=open("bank.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-GAIdulYNQ5XrHEIm72QNAzmv', bytes=3864, created_at=1711949375, filename='bank.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [26]:
# 選擇gpt-3.5-turbo model進行fine tune
client.fine_tuning.jobs.create(
  training_file="file-GAIdulYNQ5XrHEIm72QNAzmv", 
  model="gpt-3.5-turbo"
)

FineTuningJob(id='ftjob-qMnEtsgEmOUT4TAFLFdJuN5M', created_at=1711949780, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-yUQkTlPwsgEeOgYrl83isgyv', result_files=[], status='validating_files', trained_tokens=None, training_file='file-GAIdulYNQ5XrHEIm72QNAzmv', validation_file=None, user_provided_suffix=None)

In [38]:
# 改超參數
client.fine_tuning.jobs.create(
  training_file="file-GAIdulYNQ5XrHEIm72QNAzmv", 
  model="gpt-3.5-turbo", 
  hyperparameters={
    "n_epochs":20
  }
)

FineTuningJob(id='ftjob-tjNE84jKqMX0HxCFz1f6auRa', created_at=1711950688, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=20, batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-yUQkTlPwsgEeOgYrl83isgyv', result_files=[], status='validating_files', trained_tokens=None, training_file='file-GAIdulYNQ5XrHEIm72QNAzmv', validation_file=None, user_provided_suffix=None)

In [44]:
# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve("ftjob-tjNE84jKqMX0HxCFz1f6auRa")

FineTuningJob(id='ftjob-tjNE84jKqMX0HxCFz1f6auRa', created_at=1711950688, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=20, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-yUQkTlPwsgEeOgYrl83isgyv', result_files=[], status='running', trained_tokens=None, training_file='file-GAIdulYNQ5XrHEIm72QNAzmv', validation_file=None, user_provided_suffix=None)

In [46]:
# 列出 10 個來自微調作業的事件
client.fine_tuning.jobs.list_events(fine_tuning_job_id="ftjob-tjNE84jKqMX0HxCFz1f6auRa", limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-nGHZqrkq8Ee28s2hVYADzSXb', created_at=1711950986, level='info', message='Step 71/220: training loss=1.09', object='fine_tuning.job.event', data={'step': 71, 'train_loss': 1.0946775674819946, 'total_steps': 220, 'train_mean_token_accuracy': 0.7227723002433777}, type='metrics'), FineTuningJobEvent(id='ftevent-NNt9Q8jN0udkay06zq7I5wal', created_at=1711950959, level='info', message='Step 61/220: training loss=0.71', object='fine_tuning.job.event', data={'step': 61, 'train_loss': 0.7129967212677002, 'total_steps': 220, 'train_mean_token_accuracy': 0.761904776096344}, type='metrics'), FineTuningJobEvent(id='ftevent-VmvtHeNDVuk5cDnMs58GMxIu', created_at=1711950931, level='info', message='Step 51/220: training loss=0.54', object='fine_tuning.job.event', data={'step': 51, 'train_loss': 0.5435289144515991, 'total_steps': 220, 'train_mean_token_accuracy': 0.8113207817077637}, type='metrics'), FineTuningJobEvent(id='ftevent-ce

## 比較一般的 gpt-3.5-turbo 跟 fine-tuning 後的model

In [8]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "你是使用繁體中文的銀行客服人員"},
    {"role": "user", "content": "晶片密碼輸入錯誤鎖卡怎麼辦?"}
  ]
)

In [9]:
print(completion.choices[0].message.content)

如果您的晶片密碼輸入錯誤導致銀行卡被鎖卡，您可以通過以下方式解鎖：

1. 聯繫您的銀行客服。您可以撥打銀行的客服電話號碼，向客服人員報告問題並請求協助解鎖您的銀行卡。

2. 至銀行分行辦理。您也可以親自前往最近的銀行分行，與銀行職員溝通並提交申請進行解鎖手續。

無論使用哪種方式，請準備好相應的身份證明文件以便確認您的身份。在解鎖銀行卡後，請注意謹慎管理您的晶片密碼，避免再次輸入錯誤而導致鎖卡情況發生。


In [10]:
completion = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-0125:personal::9953NgaB",
  messages=[
    {"role": "system", "content": "你是使用繁體中文的銀行客服人員"},
    {"role": "user", "content": "晶片密碼輸入錯誤鎖卡怎麼辦?"}
  ]
)

In [11]:
print(completion.choices[0].message.content)

本人攜帶身份證.原留印鑑及本行金融卡至本行分行辦理密碼解鎖.


## 創建 chatbot

In [13]:
from openai import OpenAI
# import gradio

# OpenAI Key
api_key = "OPENAI_API_KEY"

client = OpenAI(
    api_key = api_key
)

history = []

system_msg = {
    "role":"system",
    "content":"你是使用繁體中文的銀行客服人員"
}
history.append(system_msg)

def ask(question):
    question_msg = {
        "role":"user",
        "content": question
    }
    history.append(question_msg)

    response = client.chat.completions.create(
    messages = history,
    model = "ft:gpt-3.5-turbo-0125:personal::9953NgaB"
    )
    answer_msg = response.choices[0].message
    history.append(answer_msg)
    return answer_msg.content

while True:
    question = input("You: ")

    if(question.strip() == ''):
        break

    answer = ask(question)
    print('\n')
    print(f"Bot: {answer}")
    print('\n')

# demo = gradio.Interface(fn=ask, inputs="text", outputs="text", title="銀行AI客服")

# demo.launch(share=True)

You:  存摺遺失如何處理?




Bot: 存摺遺失無法掛失掛失,請本人攜帶身分證正本及原留印鑑向本行最接近您的分行辦理.




You:  晶片密碼輸入錯誤鎖卡怎麼辦?




Bot: 晶片密碼輸入3次錯誤即會鎖卡,請本人攜帶身分證正本及原留印鑑至本行任一分行申請密碼解鎖.




You:  
